In [1]:
%pylab inline
import numpy as np
import warnings

from scipy import sparse

from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.svm import SVC

In [3]:
from sklearn.model_selection import cross_val_score

In [4]:
from sklearn.linear_model import Lasso

In [5]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15, 8)

In [6]:
def read_data_from_file(filename, shape):
    values = list()
    rows = list()
    cols = list()
    
    header = True
    for line in open(filename):
        if header:
            header = False
            continue
        row, col, value = [x for x in line.strip().split(',')]
        row, col = int(row), int(col)
        value = float(value)
        row -= 1
        col -= 1
        values.append(value)
        rows.append(row)
        cols.append(col)
        
    return sparse.csr_matrix((values, (rows, cols)), shape=shape)

In [7]:
X_train = read_data_from_file('X_train.csv', (15000, 30000)).astype(float)
X_test = read_data_from_file('X_test.csv', (15000, 30000)).astype(float)
print (X_train.shape, X_test.shape)

(15000, 30000) (15000, 30000)


In [8]:
from sklearn.preprocessing import scale

X_all = sparse.vstack([X_train, X_test])
X_all = scale(X_all, with_mean=False)
X_train = X_all[:15000, :]
X_test = X_all[15000:, :]

del X_all

In [9]:
def read_labels_from_file(filename, shape):
    labels = np.zeros(shape).astype(int)

    header = True
    for line in open(filename):
        if header:
            header = False
            continue
        row, indeces = line.strip().split(',')
        row = int(row) - 1
        indeces = [int(x) - 1 for x in indeces.split()]
        labels[row, indeces] = 1
    
    return labels

In [10]:
y_train = read_labels_from_file('y_train.csv', (15000, 98))


In [11]:
def write_labels_to_file(labels, filename):
    outfile = open(filename, 'w')
    print("Id,Labels", file=outfile)
    for i, line in enumerate(labels):
        elements = [str(x) for x in list(nonzero(line)[0] + 1)]
        
        print("%d,%s" % (i + 1, ' '.join(elements)), file=outfile)

## Моё решение (как Бейзлайн)

In [12]:
def classify_one_label(X_train, y_train, X_test, classifier=LogisticRegression()):
    classifier.fit(X_train, y_train)
    return classifier.predict(X_test)

In [13]:
y_test = np.empty(shape=(X_test.shape[0], 0))

In [17]:
for i in range(y_train.shape[1]):
    y_test = np.append(y_test, classify_one_label(
            X_train, y_train[:, i], X_test, classifier=SVC(C=1)).reshape(y_test.shape[0], 1), axis=1)
    if (i % 5 == 0 or i < 5): 
        print(i)

0
1
2
3
4
5
10
15
20


KeyboardInterrupt: 

In [16]:
write_labels_to_file(y_test, 'y_test.csv')

## Здесь заменим логрег на SVM

In [19]:
params = {'estimator__C': np.logspace(-4, 1, 5)},

clf = OneVsRestClassifier(SVC())

grid_searcher = GridSearchCV(
        clf,
        params,
        scoring=make_scorer(f1_score, average='samples'),
        cv=5
)

grid_searcher.fit(X_train, y_train)

KeyboardInterrupt: 